# 文本相似度实例

## Step1 导入相关包

In [1]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from datasets import load_dataset

## Step2 加载数据集

In [2]:
dataset = load_dataset("json", data_files="./train_pair_1w.json", split="train")
dataset

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset json downloaded and prepared to C:/Users/yuyao/.cache/huggingface/datasets/json/default-6c4b70959665864d/0.0.0/e347ab1c932092252e717ff3f949105a4dd28b27e842dd53157d2f72e276c2e4. Subsequent calls will reuse this data.


Dataset({
    features: ['sentence1', 'sentence2', 'label'],
    num_rows: 10000
})

In [4]:
dataset[0]

{'sentence1': '找一部小时候的动画片', 'sentence2': '求一部小时候的动画片。谢了', 'label': '1'}

## Step3 划分数据集

In [3]:
datasets = dataset.train_test_split(test_size=0.2)
datasets

DatasetDict({
    train: Dataset({
        features: ['sentence1', 'sentence2', 'label'],
        num_rows: 8000
    })
    test: Dataset({
        features: ['sentence1', 'sentence2', 'label'],
        num_rows: 2000
    })
})

## Step4 数据集预处理

In [19]:
import torch

tokenizer = AutoTokenizer.from_pretrained("hfl/chinese-macbert-base")

def process_function(examples):
    tokenized_examples = tokenizer(examples["sentence1"], examples["sentence2"], max_length=128, truncation=True)
    tokenized_examples["labels"] = [float(label) for label in examples["label"]]
    return tokenized_examples

tokenized_datasets = datasets.map(process_function, batched=True, remove_columns=datasets["train"].column_names)
tokenized_datasets

Map:   0%|          | 0/8000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 8000
    })
    test: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 2000
    })
})

In [20]:
print(tokenized_datasets["train"][0])

{'input_ids': [101, 1062, 4265, 1920, 782, 8024, 1963, 3362, 2769, 1762, 6878, 1168, 2600, 1385, 808, 1184, 6878, 1168, 4640, 2370, 7363, 678, 8024, 6929, 6421, 2582, 720, 1215, 8043, 102, 800, 2697, 6230, 2533, 800, 2190, 6821, 5439, 1928, 2094, 3683, 2190, 800, 1520, 1520, 6820, 779, 8024, 4507, 754, 800, 2190, 6821, 702, 782, 772, 4495, 4638, 3946, 2658, 679, 4881, 2544, 5010, 6629, 3341, 511, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'labels': 0.0}


## Step5 创建模型

In [21]:
from transformers import BertForSequenceClassification 
model = AutoModelForSequenceClassification.from_pretrained("hfl/chinese-macbert-base", num_labels=1)

Some weights of the model checkpoint at hfl/chinese-macbert-base were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not i

## Step6 创建评估函数

In [8]:
import evaluate

acc_metric = evaluate.load("accuracy")
f1_metirc = evaluate.load("f1")

In [22]:
def eval_metric(eval_predict):
    predictions, labels = eval_predict
    predictions = [int(p > 0.5) for p in predictions]
    labels = [int(l) for l in labels]
    # predictions = predictions.argmax(axis=-1)
    acc = acc_metric.compute(predictions=predictions, references=labels)
    f1 = f1_metirc.compute(predictions=predictions, references=labels)
    acc.update(f1)
    return acc

## Step7 创建TrainingArguments

In [23]:
train_args = TrainingArguments(output_dir="./cross_model",      # 输出文件夹
                               per_device_train_batch_size=32,  # 训练时的batch_size
                               per_device_eval_batch_size=32,  # 验证时的batch_size
                               logging_steps=10,                # log 打印的频率
                               evaluation_strategy="epoch",     # 评估策略
                               save_strategy="epoch",           # 保存策略
                               save_total_limit=3,              # 最大保存数
                               learning_rate=2e-5,              # 学习率
                               weight_decay=0.01,               # weight_decay
                               metric_for_best_model="f1",      # 设定评估指标
                               load_best_model_at_end=True)     # 训练完成后加载最优模型
train_args

TrainingArguments(
_n_gpu=1,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_pin_memory=True,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
ddp_timeout=1800,
debug=[],
deepspeed=None,
disable_tqdm=False,
do_eval=True,
do_predict=False,
do_train=False,
eval_accumulation_steps=None,
eval_delay=0,
eval_steps=None,
evaluation_strategy=epoch,
fp16=False,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
fsdp=[],
fsdp_config={'fsdp_min_num_params': 0, 'xla': False, 'xla_fsdp_grad_ckpt': False},
fsdp_min_num_params=0,
fsdp_transformer_layer_cls_to_wrap=None,
full_determinism=False,
gradient_accumulation_steps=1,
gradient_checkpointing=False,
greater_is_better=True,
group_by_length=False,
half_precision_backend=auto,
hub_model_id=None,
hub_private_repo=False,
hub_strategy=every_save,
hub_token=<HUB_TOKEN>,
ign

## Step8 创建Trainer

In [24]:
from transformers import DataCollatorWithPadding
trainer = Trainer(model=model, 
                  args=train_args, 
                  train_dataset=tokenized_datasets["train"], 
                  eval_dataset=tokenized_datasets["test"], 
                  data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
                  compute_metrics=eval_metric)

## Step9 模型训练

In [25]:
trainer.train()

c:\Users\yuyao\miniconda3\envs\transformers\lib\site-packages\transformers\optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


  0%|          | 0/750 [00:00<?, ?it/s]

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


{'loss': 0.2645, 'learning_rate': 1.9733333333333336e-05, 'epoch': 0.04}
{'loss': 0.1622, 'learning_rate': 1.9466666666666668e-05, 'epoch': 0.08}
{'loss': 0.1353, 'learning_rate': 1.9200000000000003e-05, 'epoch': 0.12}
{'loss': 0.1558, 'learning_rate': 1.8933333333333334e-05, 'epoch': 0.16}
{'loss': 0.1547, 'learning_rate': 1.866666666666667e-05, 'epoch': 0.2}
{'loss': 0.1287, 'learning_rate': 1.8400000000000003e-05, 'epoch': 0.24}
{'loss': 0.1432, 'learning_rate': 1.8133333333333335e-05, 'epoch': 0.28}
{'loss': 0.1323, 'learning_rate': 1.7866666666666666e-05, 'epoch': 0.32}
{'loss': 0.1278, 'learning_rate': 1.76e-05, 'epoch': 0.36}
{'loss': 0.1369, 'learning_rate': 1.7333333333333336e-05, 'epoch': 0.4}
{'loss': 0.1196, 'learning_rate': 1.706666666666667e-05, 'epoch': 0.44}
{'loss': 0.1168, 'learning_rate': 1.6800000000000002e-05, 'epoch': 0.48}
{'loss': 0.1455, 'learning_rate': 1.6533333333333333e-05, 'epoch': 0.52}
{'loss': 0.1418, 'learning_rate': 1.6266666666666668e-05, 'epoch': 0.

  0%|          | 0/63 [00:00<?, ?it/s]

...

  0%|          | 0/63 [00:00<?, ?it/s]

{'eval_loss': 0.08298075199127197, 'eval_accuracy': 0.8965, 'eval_f1': 0.8654970760233919, 'eval_runtime': 1.8083, 'eval_samples_per_second': 1105.987, 'eval_steps_per_second': 34.839, 'epoch': 2.0}
{'loss': 0.0717, 'learning_rate': 6.4000000000000006e-06, 'epoch': 2.04}
{'loss': 0.059, 'learning_rate': 6.133333333333334e-06, 'epoch': 2.08}
{'loss': 0.0697, 'learning_rate': 5.8666666666666675e-06, 'epoch': 2.12}
{'loss': 0.0578, 'learning_rate': 5.600000000000001e-06, 'epoch': 2.16}
{'loss': 0.0716, 'learning_rate': 5.333333333333334e-06, 'epoch': 2.2}
{'loss': 0.0573, 'learning_rate': 5.0666666666666676e-06, 'epoch': 2.24}
{'loss': 0.0645, 'learning_rate': 4.800000000000001e-06, 'epoch': 2.28}
{'loss': 0.0512, 'learning_rate': 4.533333333333334e-06, 'epoch': 2.32}
{'loss': 0.0653, 'learning_rate': 4.266666666666668e-06, 'epoch': 2.36}
{'loss': 0.0548, 'learning_rate': 4.000000000000001e-06, 'epoch': 2.4}
{'loss': 0.0453, 'learning_rate': 3.7333333333333337e-06, 'epoch': 2.44}
{'loss':

  0%|          | 0/63 [00:00<?, ?it/s]

{'eval_loss': 0.07563769817352295, 'eval_accuracy': 0.911, 'eval_f1': 0.8888888888888888, 'eval_runtime': 1.8058, 'eval_samples_per_second': 1107.519, 'eval_steps_per_second': 34.887, 'epoch': 3.0}
{'train_runtime': 86.1781, 'train_samples_per_second': 278.493, 'train_steps_per_second': 8.703, 'train_loss': 0.0897649096250534, 'epoch': 3.0}


TrainOutput(global_step=750, training_loss=0.0897649096250534, metrics={'train_runtime': 86.1781, 'train_samples_per_second': 278.493, 'train_steps_per_second': 8.703, 'train_loss': 0.0897649096250534, 'epoch': 3.0})

## Step10 模型评估

In [26]:
trainer.evaluate(tokenized_datasets["test"])

  0%|          | 0/63 [00:00<?, ?it/s]

{'eval_loss': 0.07563769817352295,
 'eval_accuracy': 0.911,
 'eval_f1': 0.8888888888888888,
 'eval_runtime': 1.8573,
 'eval_samples_per_second': 1076.813,
 'eval_steps_per_second': 33.92,
 'epoch': 3.0}

## Step11 模型预测

In [14]:
from transformers import pipeline, TextClassificationPipeline

In [27]:
model.config.id2label = {0: "不相似", 1: "相似"}

In [28]:
pipe = pipeline("text-classification", model=model, tokenizer=tokenizer, device=0)

In [35]:
result = pipe({"text": "我喜欢北京", "text_pair": "天气怎样"}, function_to_apply="none")
result["label"] = "相似" if result["score"] > 0.5 else "不相似"
result

{'label': '不相似', 'score': 0.054742373526096344}